'**All models are wrong, but some are useful**' - George E.P. Box, British Statistician

This notebook is on House prices prediction using linear regression. 
Here in this notebook, I have started off taking some parts  from the amazing notebooks by  [Comprehensive data analysis by Pedro](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python) and [Stacked Regressions by Serigne](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard)

**Focus of the notebook**
The notebook reproduces some of the results from above mentioned notebooks. But takes a differente standpoint regarding the assumptions of linear regression. The focus of this notebook is to illustrate the assumptions of linear regression in a methodological way.

**Assumptions of linear regression:**
1. Linearity- 
2. Normality
3. Homoscedasticity
4. Independence( No Multicollinearity)


Some notebooks consider the transformations well ahead of the modelling. For e.g. [in  Notebook by Serigne](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard?rvi=1) and also [Notebook by Pedro](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python) . However, in this notebook, tranformation have not been considered aprior to modelling to prevent the misconception that can arise to reader on assumptions of Linear regression.

Infact when I googled, came to know that there are already lot of intersting discussions on this very topic about the normality assumption in linear regression. For e.g. [ in this stackexchange discussion ](https://stats.stackexchange.com/questions/12262/what-if-residuals-are-normally-distributed-but-y-is-not?newreg=be2fc14894c247fd95e8abdbb16f5af0).
Also the website [by Robert Nau of Duke University](https://people.duke.edu/~rnau/testing.htm) helped me understand some key concepts.

The conclusions (based on my understanding) are as follows:
1. The normality assumption in linear regression is for the residuals and not for the response/predictor variable. 
2. The normality assumption of residuals enables  one to calculate p-values and make statistical inferences. Because the p-value, confidence intervals are based on this assumption that residuals are normally distributed.
3. If someone doesn't want to make any statistical inferences after linear regression and the residuals are not normally distributed, it is perfectly valid.

As an additional point,even if the distribution of predictor variables is non-normal , Central limit theorem guarantees that the resulting  distribution will be normal i.e. the response variable which is being modelled as linear combination of predictor variables(could be Non-normal) will turn out to be normal.
Now here is the catch. What if the resulting distribution of response variable y(SalePrice here) doesn't follow normal distribution?  Let us go ahead with the data analysis and check what happens.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# use the inline backend to generate the plots within the browser
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')
%matplotlib inline
mpl.style.use('ggplot')  # optional: for ggplot-like style

# check for latest version of Matplotlib
print('Matplotlib version: ', mpl.__version__) # >= 2.0.0
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Loading data and defining missing values: **

The pandas library read_csv  by default searches for the following missing value types:
 ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’
 There is an additional argument **"na_values= "** that can be passed so as to check custom na type values. for e.g. a space character " " etc.

In [ ]:
missing_values = ["n/a", "na", "--"," "]
train= pd.read_csv('../input/housepricesadvanced-regression-techniques/train (1).csv',na_values = missing_values)
test= pd.read_csv('../input/housepricesadvanced-regression-techniques/test (1).csv',na_values = missing_values)
train.shape

What are the predictor variables we  have in our dataset?
[Ames dataset](http://jse.amstat.org/v19n3/decock.pdf) has great details on how the data has been configured and other interesting details. Intrested reader may want to look into the documentation of the data set. As a word of caution, some of the links in the document mentioned above seems inactive. 
But this link provides nice [Definition of data set variables](https://cran.r-project.org/web/packages/AmesHousing/AmesHousing.pdf)

In [ ]:
train.columns

'SalePrice ' is the response variable we want to predict.Let's look at its statistical summary

In [ ]:
train['SalePrice'].describe()

In [ ]:
sns.distplot(train['SalePrice']);

In [ ]:
scatter_data=pd.concat([train['SalePrice'], train['GrLivArea']], axis=1)
scatter_data.head()

To understand how sales price changes with respect to predictor variables. Out of many variables we can choose to illustrate how 

In [ ]:
scatter_data.plot(kind='scatter', x='GrLivArea', y='SalePrice', figsize=(10, 6), color='darkblue')
plt.title('sale price vs grlivarea')
plt.xlabel('grlivarea')
plt.ylabel('sale price ')

plt.show()

In [ ]:
scatter_data=pd.concat([train['SalePrice'], train['TotalBsmtSF']], axis=1)
scatter_data.head()

In [ ]:
scatter_data.plot(kind='scatter', x='TotalBsmtSF', y='SalePrice', figsize=(10, 6), color='darkblue')

plt.title('sale price vs grlivarea')
plt.xlabel('TotalBsmtSF')
plt.ylabel('sale price ')

plt.show()

In [ ]:
var = 'OverallQual'
data_box = pd.concat([train['SalePrice'], train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data_box)
fig.axis(ymin=0, ymax=800000);

In [ ]:
var = 'YearBuilt'
data_box = pd.concat([train['SalePrice'], train[var]], axis=1)
f, ax = plt.subplots(figsize=(18, 12))
fig = sns.boxplot(x=var, y="SalePrice", data=data_box)
fig.axis(ymin=0, ymax=800000);

In [ ]:
corrmat=train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm=train[cols].corr()
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True,annot=True, square=True, fmt='.2f', annot_kws={'size': 10})


In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], size = 2.5)
plt.show();

In [ ]:
#saleprice_scaled = StandardScaler().fit_transform(data['SalePrice'][:,np.newaxis])

In [ ]:
#saleprice_scaled2 = np.delete(saleprice_scaled, np.where(
#    (saleprice_scaled >=3))[0], axis=0)
#saleprice_scaled2[saleprice_scaled2>=3]

In [ ]:
#bivariate analysis saleprice/grlivarea
var = 'GrLivArea'
data1 = pd.concat([train['SalePrice'], train[var]], axis=1)
data1.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
train.sort_values(by = 'GrLivArea', ascending = False)[:2]


In [ ]:
train= train.drop(train[train['Id'] == 1298].index)
train= train.drop(train[train['Id'] == 523].index)

In [ ]:
sns.distplot(train['SalePrice'],fit= norm);
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)

In [ ]:
#applying log transformation
#train['SalePrice'] = np.log1p(train['SalePrice'])
#transformed histogram and normal probability plot
sns.distplot(train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)

In [ ]:
#histogram and normal probability plot
#sns.distplot(data['GrLivArea'], fit=norm);
#fig = plt.figure()
#res = stats.probplot(data['GrLivArea'], plot=plt)

In [ ]:
data=pd.concat((train, test))
y= train.SalePrice.values
data.drop(columns=['SalePrice'], inplace= True)
data.shape

In [ ]:
data.shape

In [ ]:
y= train.SalePrice.values
y

In [ ]:
#applying log transformation
#data['GrLivArea'] = np.log(data['GrLivArea'])
#transformed histogram and normal probability plot
sns.distplot(data['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(data['GrLivArea'], plot=plt)

In [ ]:
#histogram and normal probability plot
sns.distplot(data['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(data['TotalBsmtSF'], plot=plt)

In [ ]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

**Missing value treatment**

**PoolQC** : data description says NA means "No Pool"

**MiscFeature**  : data description says NA means "no misc feature"

**Alley** : data description says NA means "no alley access"

**FireplaceQu** : data description says NA means "no fireplace"

**Fence** : data description says NA means "no fence"


In [ ]:
data["PoolQC"] =data["PoolQC"].fillna("None")
data["MiscFeature"] = data["MiscFeature"].fillna("None")
data["Alley"] = data["Alley"].fillna("None")
data["Fence"]= data["Fence"].fillna("None")
data["FireplaceQu"]= data["FireplaceQu"].fillna("None")





In [ ]:
data.columns
data.shape

It is assumed that  similar neighborhoods will be similar fromtage. Hence the median of frontage corresponding to a neighborhood has been used to imputed the missing value.

In [ ]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
data["LotFrontage"] = data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

For the columns "GarageType","GarageFinish",'GarageQual', 'GarageCond' missing value implies the feature not being available. Hence considered as None". Will be later encoded as cetegorical using onehot encoding.

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    data[col] =data[col].fillna('None')

**GarageYrBlt, GarageArea and GarageCars** : Replacing missing data with 0 (Since No garage = no cars in such garage.)

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    data[col] = data[col].fillna(0)

**BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBath** : missing values are likely zero for having no basement

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    data[col] = data[col].fillna(0)

**BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 and BsmtFinType2 :** For all these categorical basement-related features, NaN means that there is no basement.

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    data[col] = data[col].fillna('None')

**MasVnrArea and MasVnrType** : NA most likely means no masonry veneer for these houses. We can fill 0 for the area and None for the type.

In [ ]:
data["MasVnrType"] = data["MasVnrType"].fillna("None")
data["MasVnrArea"] = data["MasVnrArea"].fillna(0)

**MSZoning (The general zoning classification)** : 'RL' is by far the most common value. So we can fill in missing values with 'RL'

In [ ]:
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode()[0])

**Utilities** : For this categorical feature all records are "AllPub", except for one "NoSeWa" and 2 NA . Since the house with 'NoSewa' is in the training set, this feature won't help in predictive modelling. We can then safely remove it.

In [ ]:
data = data.drop(['Utilities'], axis=1)


In [ ]:
data.shape

**Functional** : data description says NA means typical

In [ ]:
data["Functional"] =data["Functional"].fillna("Typ")

**Electrical** : It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.

In [ ]:
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])

**KitchenQual**: Only one NA value, and same as Electrical, we set 'TA' (which is the most frequent) for the missing value in KitchenQual.

In [ ]:
data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])

**Exterior1st and Exterior2nd** : Again Both Exterior 1 & 2 have only one missing value. We will just substitute in the most common string

In [ ]:
data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])

**SaleType** : Fill in again with most frequent which is "WD"

In [ ]:
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])

**MSSubClass** : Na most likely means No building class. We can replace missing values with None

In [ ]:
data['MSSubClass'] = data['MSSubClass'].fillna("None")

Let's onfirm if there are still any missing values:

In [ ]:
data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :data_na})
missing_data.head()

Some categorical variables are represented in numerical variables. So lets transform them.

In [ ]:
#MSSubClass=The building class
data['MSSubClass'] = data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
data['OverallCond'] = data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
data['YrSold'] = data['YrSold'].astype(str)
data['MoSold'] = data['MoSold'].astype(str)


Converting ordinal variables to preserve the order of the value. In other words, the quantitative values of variables increase corresponding to the variables' qualitative interpretation. For example for the variable, Kitchen quality, the label encoding can be custommised as shown below as suggested by [Koji ](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard/comments)

In [ ]:
#mapper = {'Ex': 4, 'Gd': 3, 'TA':2, 'Fa': 1}
#data['KitchenQual'] = data['KitchenQual'].map(mapper)


If we implement this for all such variables where, label encoding is appropriate than onehot encoding, [in this reference by mitra mirshafiee ](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard/comments)

from sklearn.preprocessing import OrdinalEncoder
ord_fields=['MSSubClass','ExterQual','LotShape','BsmtQual','BsmtCond','BsmtExposure',
            'BsmtFinType1', 'BsmtFinType2','HeatingQC','Functional',
            'FireplaceQu','KitchenQual', 'GarageFinish','GarageQual','GarageCond','PoolQC','Fence']
orders=[ ['20','30','40','45','50','60','70','75','80','85', '90','120','150','160','180',
    '190'],  ['Fa','TA','Gd','Ex'], ['Reg','IR1' ,'IR2','IR3'], 
 ['None','Fa','TA','Gd','Ex'], ['None','Po','Fa','TA','Gd','Ex'],
 ['None','No','Mn','Av','Gd'],['None','Unf','LwQ', 'Rec','BLQ','ALQ' , 
'GLQ' ], ['None','Unf','LwQ', 'Rec','BLQ','ALQ' , 'GLQ' ],
 ['Po','Fa','TA','Gd','Ex'],  ['Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'],
 ['None','Po','Fa','TA','Gd','Ex'],  ['Fa','TA','Gd','Ex'],
 ['None','Unf','RFn','Fin'],  ['None','Po','Fa','TA','Gd','Ex'],
['None','Po','Fa','TA','Gd','Ex'],
 ['None','Fa','Gd','Ex'], ['None','MnWw','GdWo','MnPrv','GdPrv'] ]
for i in range(len(orders)):     
    ord_en=OrdinalEncoder(categories = {0:orders[i]}) 
    data.loc[:,ord_fields[i]]=ord_en.fit_transform(data.loc[:,ord_fields[i]].values.reshape(-1,1))


In [ ]:
print('Shape all_data: {}'.format(data.shape))

We can go with the label encoding for the other features that do not require ordinal encoding.
Features: 

In [ ]:
from sklearn.preprocessing import LabelEncoder
#cols = ( 'LandSlope','LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 
   #     'YrSold', 'MoSold')
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(data[c].values)) 
    data[c] = lbl.transform(list(data[c].values))

# shape        
print('Shape all_data: {}'.format(data.shape))

In [ ]:
c=data.dtypes
c.head(20)

We can drop the column Id so as to ignore it during modelling

In [ ]:
data.drop(columns='Id', inplace= True)
data.shape

In [ ]:
# Adding total sqfootage feature 
data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']
data.shape

Let us proceed with the one hot encoding for all categorical features. pd.get_dummies , by default  creates one hot encoding for all variables of the class 'object'.

In [ ]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics
numeric_feats = data.dtypes[data.dtypes != "object"].index
# Check the skew of all numerical features
skewed_feats = data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


In [ ]:
skewness.head(59)

In [ ]:
skew(data['Alley'])

In [ ]:
skewness.index

Let us save the data as 'data_X' before transformations and other manipulation tasks. This data_X will be used later so as to check the linear regression assumptions, for which its good to have the non-transformed data.

In [ ]:
data_X=data

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to  log transform".format(skewness.shape[0]))

#from scipy.special import boxcox1p
skewed_features = skewness.index
skewed_features
data[skewed_features]=np.log1p(data[skewed_features])

#lam = 0.15
#for feat in skewed_features:
 #    data[feat] = boxcox1p(data[feat], lam)

In [ ]:
data = pd.get_dummies(data)
print(data.shape)


    

In [ ]:
train_size= train.shape[0]
train_size

We can unmerge the data into train and test as per the size of original train and test data set that was imported in first place.

In [ ]:
X_train= data[:train_size]
y_train1= y[:train_size]
y_train = train.SalePrice.values
X_test=data[train_size:]


In [ ]:
y

In [ ]:
y_train1

In [ ]:
y_train

In [ ]:
train_size

At this point , we are ready with the train and test data sets. 

In [ ]:
X_train.shape

In [ ]:
y.shape

In [ ]:
data.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)


In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(reg, X_train, y_train, cv=3))

In [ ]:
from sklearn.linear_model import Lasso 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error


In [ ]:
X_test.columns

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
y_train

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [ ]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
print(score)

Now that data is ready, we can go ahead with the modelling. Note that we have not done any transformations on the data like box-cox transformation to normalize the skewness of the distributions in different variables. One can attempt these tranformations  if not satified with the model performance.

**Residual plot:**  Since we are done with the modelling part, let us plot the residuals and check if they are normally distributed or not

In [ ]:
y_pred=reg.predict(X_train)
res= y_train-y_pred
sns.distplot(res, fit= norm)
res=np.sort(res)
res

**Linear regression 1st assumption**:  It can be observed that the residuals are closely alligned with the normal distribution . But is it enough to assuume that normal distribution is satified? 

**Q-Q plot:** Let us go ahead and call the Q-Q plot from statsmodels.api. 

In [ ]:
import statsmodels.api as sm
sm.qqplot(res,line='45',fit=True,dist=stats.norm)
res


**Probplot**: Now there is the other version of illustrating the same point . That is by calling the probplot from ScikitLearn.

In [ ]:
residuals = stats.probplot(res, plot=plt)

The plots suggest some kind of non -linear behaviour of normalized residuals. So there is an indication that the normality assumption of Linear regression hasn't been satisfied well. Let us cplot the distribution of the response variable "SalePrice". 

In [ ]:
sns.distplot(y_pred, fit= norm)

In [ ]:
fig=sm.qqplot(y_pred,line='45',fit=True,dist=stats.norm)

According to Central Limit Theorem, the distribution should be Normal. But we can see that there are soe visible deviations from the expected behaviour. Let us consider some tranformations on respnse variable 'SalePrice'

In [ ]:
y_train_transf=np.log1p(y_train)
sns.distplot(y_train_transf, fit= norm)

In [ ]:
fig=sm.qqplot(y_train_transf,line='45',fit=True,dist=stats.norm)

We can observe that, after the transfromation, the distribution of saleprice approaches closer to normal distribution, How does this affect the prediction, Let's see.

In [ ]:
reg = LinearRegression().fit(X_train, y_train_transf)
reg.score(X_train, y_train_transf)

In [ ]:
y_pred=reg.predict(X_train)
res= y_train_transf-y_pred
sns.distplot(res, fit= norm)


In [ ]:
fig=sm.qqplot(res,line='45',fit=True,dist=stats.norm)

The distribution of residuals is comparitively more normal than the previous un-transformed version. But not close enough to be called as standard normal distribution. So let's check for the other assumptions of linear regression proceeding to **"homoscedasticity"**.


In [ ]:
##scatter_data=pd.concat(res,y_train)
#scatterdata
plt.scatter(y_pred, res)
plt.xlabel("Sale Price")
plt.ylabel("Residuals")
plt.title("Residuals Vs SalePrice(Fitted values)")
plt.show()

**Homoscedasticity :**
 From the above plot, of Residuals vs Fitted values, it appears that there is no strong sense of variation in residuals w.r.t. fitted values

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
feats=data_X.dtypes[data_X.dtypes != "object"].index
feats
data_numeric= data_X[feats]

vif_data = pd.DataFrame()
vif_data["feature"] = data_numeric.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(data_numeric.values, i)
                          for i in range(len(data_numeric.columns))]
vif_data.sort_values(by='VIF',ascending=False)


We can observe that the features 'YearBuilt' and 'YearRemododAdd' are strongly correlated. Hence we can redefine the 'YearRemodAdd' to a new variable such that the variable captures, whether remodelling has been done or not. In the dataset, If  a house is not remodelled the 'YearBuit'= 'YearRemmodAdd'. So we use that condition to extract the infomation on whether remodelling was done or not on a partiuclar house.

In [ ]:
data_numeric.loc[data_numeric['YearRemodAdd']==data_numeric['YearBuilt'],'YearRemodAdd']=0
data_numeric[['YearBuilt','YearRemodAdd']]


In [ ]:
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(data_numeric.values, i)
                          for i in range(len(data_numeric.columns))]
vif_data.sort_values(by='VIF',ascending=False)

The variable **'GrLivArea'** has vey high VIF. Possible that it is corrrleating with **'1st Flr SF'** and '**2nd FLr SF**'. When we tabulate a shown below table, it is clear that GRLivArea is nothing but sum of '1stFlrSF' '2ndFlrSF'. So we can safely remove the variable 'GrLivArea' from our data set. Also its  Total SF = BSmtSF+ 1stFlrSF+ 2ndFlrSF, we can remove that variable as well.

In [ ]:

train[['GrLivArea','1stFlrSF','2ndFlrSF']].head(20)

In [ ]:
data_numeric.drop(['1stFlrSF','2ndFlrSF'], axis=1, inplace=True)

In [ ]:
data_numeric.drop('GrLivArea', axis=1, inplace=True)
#data_numeric.drop('TotalSF', axis=1, inplace=True)

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = data_numeric.columns
vif_data["VIF"] = [variance_inflation_factor(data_numeric.values, i)
                          for i in range(len(data_numeric.columns))]
vif_data.sort_values(by='VIF',ascending=False)

The variable 'year built' is one geric variable which could encapsulate the  information about many other varibales. FOr e.g. it is seen to correlate with the sq.feet area (1stflrsf ) . Now it can be seen that it is correlating with pool quality as well. So may be it can lead us to the understandong that the older houses may have low pool quality. So lets drop the variable 'year built' and see how our VIF table changes.

In [ ]:
data_numeric.drop('YearBuilt', axis=1, inplace=True)
vif_data = pd.DataFrame()
vif_data["feature"] = data_numeric.columns
vif_data["VIF"] = [variance_inflation_factor(data_numeric.values, i)
                          for i in range(len(data_numeric.columns))]
vif_data.sort_values(by='VIF',ascending=False)

There is some improvement, still the VIFs are way too high as far as the standard acceptable values of VIF are concerned. One can make suitable analysis to overcome this problem in their further study. That is it from my end.
In summary, we have accomplished the task of how diagnoise whether a model is suitable for a given data set. Specifically, by appealing to the first principles of linear regression i.e. linearity, normality of residuals, homoscedasticity and no multi collinearity. Thanks for reading.